# ARI 정의 및 계산 방법
## ARI (Accident Risk Index) : 교통사고위험지수
- 어린이 보호구역의 무인 교통단속용 장비 설치장소 고려사항 중 하나
- 도로교통법 시행규칙에서 명시하고 있음

## ARI 계산 방법
> $$ARI = {\sqrt{KSI^2+사고건수^2} \over 년}$$ 
- KSI(Killed or Seriously Injured: 사고심각성 대표 지표): 사망자 수 + 중상자 수 (어린이 보호구역 내)
- 사고건수: 인명피해사고 건수

# ARI considering Commuting Distance
- 통학거리를 고려한 교통사고위험지수를 제안
- 통학거리를 반영하여 각 어린이보호구역 근방의 ARI를 산정하고 비교해보고자 함

## 통학거리 기준
- 초중등교육법 시행령에 따른 초등학교 통학거리는 1.5km 이내
- 서울시의 경우 인구밀집도를 고려한 초등학교 통학거리는 대부분이 1km 이내
http://plug.hani.co.kr/data/2138604

# 01 데이터 불러오기 

In [1]:
import numpy as np
import pandas as pd

In [2]:
szone = pd.read_csv('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/Data/서울시 어린이보호구역.csv', index_col = 0)
szone

,시설종류,대상시설명,행정구역,경도,위도
0,초등학교,화랑초교,노원구,127.093803,37.626918
1,초등학교,공릉초교,노원구,127.074631,37.618615
2,초등학교,중원초교,노원구,127.062768,37.643983
3,초등학교,원광초교,노원구,127.074916,37.654312
4,학원,국풍2000학원,노원구,127.076703,37.650631
...,...,...,...,...,...
1664,초등학교,서교초교,마포구,126.924378,37.554889
1665,초등학교,성산초교,마포구,126.910949,37.553410
1666,초등학교,신북초교,마포구,126.905363,37.571104
1667,초등학교,한서초교,마포구,126.949779,37.553648


In [3]:
acc_szone = pd.read_csv('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/Data/스쿨존내어린이 사고다발지역정보.csv', encoding='utf-8', index_col = 0)
acc_szone.head()

,level_0,index,사고다발지FID,연도,법정동코드,지점코드,행정구역,지점명,발생건수,사상자수,사망자수,중상자수,경상자수,부상신고자수,경도,위도,다발지역폴리곤,지역
0,0,105,6141105,2016,1126010400,11260001,중랑구,서울특별시 중랑구 묵동(묵현초교 부근),3,3,0,0,3,0,127.075014,37.608344,"{""type"":""Polygon"",""coordinates"":[[[127.0777089...",서울
1,1,106,6107962,2016,1129013800,11290001,성북구,서울특별시 성북구 장위동(장위초교 부근),3,3,0,0,3,0,127.048284,37.611186,"{""type"":""Polygon"",""coordinates"":[[[127.0509790...",서울
2,2,107,6107948,2016,1132010700,11320001,도봉구,서울특별시 도봉구 창동(창원초교 부근),3,3,0,2,1,0,127.041023,37.653416,"{""type"":""Polygon"",""coordinates"":[[[127.0437176...",서울
3,3,108,6139185,2016,1154510300,11545001,금천구,서울특별시 금천구 시흥동(문백초교 부근),1,1,1,0,0,0,126.900058,37.449438,"{""type"":""Polygon"",""coordinates"":[[[126.9027531...",서울
4,4,148,6229749,2017,1111017500,11110001,종로구,서울특별시 종로구 숭인동(창신초교 부근),2,2,0,0,1,1,127.015572,37.576610,"{""type"":""Polygon"",""coordinates"":[[[127.0182666...",서울


In [4]:
acc_zone = pd.read_csv('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/Data/어린이 사고다발지역.csv', index_col = 0)
acc_zone.head()

,법정동코드,지점코드,연도,행정구역,지점명,발생건수,사상자수,사망자수,중상자수,경상자수,부상신고자수,위도,경도,다발지역폴리곤
85,1117012900,11170001,2016,용산구,"서울특별시 용산구 이촌동(이촌1동 주민선터, 용산청소년수 부근)",3,4,0,0,2,2,37.521424,126.973010,"{""type"":""Polygon"",""coordinates"":[[[126.9748069..."
86,1123010600,11230001,2016,동대문구,서울특별시 동대문구 장안동(승용빌딩 부근),3,3,0,1,1,1,37.566494,127.068121,"{""type"":""Polygon"",""coordinates"":[[[127.0699180..."
87,1126010200,11260001,2016,중랑구,서울특별시 중랑구 상봉동(중화초교 부근),3,3,0,0,3,0,37.602417,127.094933,"{""type"":""Polygon"",""coordinates"":[[[127.0967298..."
88,1126010400,11260002,2016,중랑구,서울특별시 중랑구 묵동(묵2동사무소 부근),3,3,0,0,3,0,37.609448,127.074558,"{""type"":""Polygon"",""coordinates"":[[[127.0763544..."
89,1129013600,11290001,2016,성북구,서울특별시 성북구 하월곡동(하월곡생명중앙교회 부근),3,4,0,0,4,0,37.607851,127.036376,"{""type"":""Polygon"",""coordinates"":[[[127.0381721..."


# 02 반경 내 사고건수, KSI 산출
- 스쿨존 1km 반경 내에 있는 사고다발지역과 스쿨존내사고다발지역의 사고건수, KSI 산출

In [5]:
# 지도 상의 두 점 사이의 거리를 구하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

In [6]:
# 각 스쿨존 반경 1km내 사고건수 (사고다발지역) 
acc_cnt = []
KSI_cnt = []
for i in szone.itertuples(index=False): 
  acc = 0
  dth = 0
  inj = 0
  for row in acc_zone.itertuples(index=False):

    d = haversine_distance(i.위도, i.경도, row.위도, row.경도)
    if d <= 1:
      acc += row.발생건수
      dth += row.사망자수
      inj += row.중상자수

  acc_cnt.append(acc)
  KSI_cnt.append(dth+inj)

print(acc_cnt)
print(KSI_cnt)

[0, 0, 6, 4, 4, 4, 4, 4, 4, 4, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3, 6, 3, 7, 6, 3, 0, 6, 0, 0, 0, 3, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 4, 0, 7, 6, 0, 4, 3, 6, 0, 0, 6, 0, 0, 0, 6, 6, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 10, 10, 3, 9, 3, 10, 4, 0, 3, 0, 10, 0, 0, 3, 0, 0, 0, 0, 3, 3, 3, 0, 6, 0, 0, 3, 0, 3, 0, 3, 3, 0, 3, 6, 3, 3, 3, 0, 0, 3, 0, 4, 3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 0, 3, 0, 0, 3, 6, 0, 3, 3, 0, 3, 0, 0, 3, 3, 3, 3, 6, 3, 0, 0, 0, 0, 0, 0, 6, 6, 3, 0, 0, 3, 3, 3, 6, 0, 0, 0, 3, 3, 4, 3, 3, 3, 0, 3, 6, 0, 3, 3, 0, 3, 3, 3, 0, 0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 3, 7, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 6, 0, 3, 3, 0, 0, 3, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 3, 7, 0, 7, 10, 13, 3, 4, 6, 6, 7, 13, 6, 7, 0, 7, 7, 6, 3, 0, 0, 3, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [7]:
szone['다발지역사고건수'] = acc_cnt
szone['다발지역KSI'] = KSI_cnt

In [8]:
# 각 스쿨존 반경 1km내 사고건수 (스쿨존내 사고다발지역)
acc_cnt_szone = []
KSI_cnt_szone = []
for i in szone.itertuples(index=False): 
  acc = 0
  dth = 0
  inj = 0
  for row in acc_szone.itertuples(index=False):

    d = haversine_distance(i.위도, i.경도, row.위도, row.경도)
    if d <= 1:
      acc += row.발생건수
      dth += row.사망자수
      inj += row.중상자수

  acc_cnt_szone.append(acc)
  KSI_cnt_szone.append(dth+inj)

print(acc_cnt_szone)
print(KSI_cnt_szone)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 3, 5, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 7, 2, 7, 0, 2, 5, 0, 0, 0, 2, 2, 0, 2, 2, 0, 4, 0, 0, 7, 2, 0, 5, 2, 2, 5, 2, 7, 0, 2, 0, 2, 5, 5, 0, 2, 0, 2, 2, 0, 0, 2, 7, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 5, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2, 7, 4, 0, 0, 2, 0, 7, 2, 0, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 5, 2, 0, 2, 7, 4, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2, 3, 0, 1, 1, 2, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 

In [9]:
szone['스쿨존사고건수'] = acc_cnt_szone
szone['스쿨존KSI'] = KSI_cnt_szone

In [10]:
szone.head()

,시설종류,대상시설명,행정구역,경도,위도,다발지역사고건수,다발지역KSI,스쿨존사고건수,스쿨존KSI
0,초등학교,화랑초교,노원구,127.093803,37.626918,0,0,0,0
1,초등학교,공릉초교,노원구,127.074631,37.618615,0,0,0,0
2,초등학교,중원초교,노원구,127.062768,37.643983,6,1,0,0
3,초등학교,원광초교,노원구,127.074916,37.654312,4,2,0,0
4,학원,국풍2000학원,노원구,127.076703,37.650631,4,2,0,0


# 03 ARI 산출 - 스쿨존

In [11]:
# ARI 산출 함수 정의
def ari_calc(KSI, acc_cnt):
  ARI = (np.sqrt(KSI**2 + acc_cnt**2))/5
  return np.round(ARI, 2)

In [12]:
ari = []
for row in szone.itertuples(index=False):

  sum_KSI = row.다발지역KSI + row.스쿨존KSI
  sum_acc = row.다발지역사고건수 + row.스쿨존사고건수
  res = ari_calc(sum_KSI, sum_acc)
  ari.append(res)

In [13]:
szone['ARI'] = ari

In [14]:
szone

,시설종류,대상시설명,행정구역,경도,위도,다발지역사고건수,다발지역KSI,스쿨존사고건수,스쿨존KSI,ARI
0,초등학교,화랑초교,노원구,127.093803,37.626918,0,0,0,0,0.00
1,초등학교,공릉초교,노원구,127.074631,37.618615,0,0,0,0,0.00
2,초등학교,중원초교,노원구,127.062768,37.643983,6,1,0,0,1.22
3,초등학교,원광초교,노원구,127.074916,37.654312,4,2,0,0,0.89
4,학원,국풍2000학원,노원구,127.076703,37.650631,4,2,0,0,0.89
...,...,...,...,...,...,...,...,...,...,...
1664,초등학교,서교초교,마포구,126.924378,37.554889,0,0,0,0,0.00
1665,초등학교,성산초교,마포구,126.910949,37.553410,0,0,0,0,0.00
1666,초등학교,신북초교,마포구,126.905363,37.571104,3,1,0,0,0.63
1667,초등학교,한서초교,마포구,126.949779,37.553648,0,0,0,0,0.00


In [15]:
szone.sort_values('ARI', ascending=False)

,시설종류,대상시설명,행정구역,경도,위도,다발지역사고건수,다발지역KSI,스쿨존사고건수,스쿨존KSI,ARI
1109,유치원,화일초등학교병설유치원,강서구,126.847465,37.540712,14,6,5,1,4.05
1108,초등학교,화일초등학교,강서구,126.847465,37.540712,14,6,5,1,4.05
306,어린이집,강동무지개어린이집,강동구,127.140664,37.544605,13,4,3,2,3.42
1130,유치원,예은유치원,강서구,126.847243,37.538834,14,6,2,0,3.42
300,어린이집,라온어린이집,강동구,127.139380,37.543138,13,4,2,1,3.16
...,...,...,...,...,...,...,...,...,...,...
851,어린이집,능동꿈맞이어린이집,광진구,127.085914,37.552830,0,0,0,0,0.00
852,특수학교,재한몽골학교,광진구,127.098237,37.545515,0,0,0,0,0.00
853,특수학교,광진학교,광진구,127.091562,37.537667,0,0,0,0,0.00
854,초등학교,추계초등학교,서대문구,126.954059,37.563150,0,0,0,0,0.00


In [22]:
szone_ari = szone.sort_values('ARI', ascending=False)
szone_ari.head(30)

,시설종류,대상시설명,행정구역,경도,위도,다발지역사고건수,다발지역KSI,스쿨존사고건수,스쿨존KSI,ARI
1109,유치원,화일초등학교병설유치원,강서구,126.847465,37.540712,14,6,5,1,4.05
1108,초등학교,화일초등학교,강서구,126.847465,37.540712,14,6,5,1,4.05
306,어린이집,강동무지개어린이집,강동구,127.140664,37.544605,13,4,3,2,3.42
1130,유치원,예은유치원,강서구,126.847243,37.538834,14,6,2,0,3.42
300,어린이집,라온어린이집,강동구,127.139380,37.543138,13,4,2,1,3.16
340,어린이집,한양어린이집,강동구,127.142815,37.543546,13,4,2,1,3.16
555,초등학교,천호초등학교,강동구,127.136592,37.547719,13,2,1,1,2.86
1099,어린이집,텀블랜드,서초구,127.039679,37.479240,10,6,2,1,2.78
441,어린이집,선우어린이집,강남구,127.038509,37.490733,10,5,2,1,2.68
651,어린이집,꿈나무어린이집,동작구,126.922954,37.504616,12,5,0,0,2.60


In [17]:
szone_ari.to_csv('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/구성봉/szone_ari.csv')

In [18]:
import folium

In [19]:
lat = szone['위도'].mean()
lon = szone['경도'].mean()

In [20]:
map = folium.Map(location=[lat, lon], tiles='cartodbpositron', zoom_start=11)

In [21]:
for i in acc_zone.index:
  mk_acc_zone = folium.Circle(
      [acc_zone['위도'][i], acc_zone['경도'][i]], radius=5, color='#fdae6b', popup='test', fill=True, fill_color='#fee6ce'
      )
  mk_acc_zone.add_to(map)
map